In [ ]:
import glob
import pandas as pd

In [ ]:
def find_injection_candidates(injections_df, posterior_feather_filepath, candidates_feather_output_dir, snr_cut=None):
    """
    Find the injection candidates in the posterior samples.
    Uses frequency range of the posterior samples to reduce the total number of injections to search through.

    Parameters
    ----------
    injections_df : pandas.DataFrame
        DataFrame containing the injections.

    posterior_feather_filepath : str
        Path to the injections in feather format.

    posterior_feather_filepath : str
        Path to the posterior samples in feather format.

    Returns
    -------
    injection_candidates : pandas.DataFrame
        DataFrame containing the injection candidates.
    """
    # Load posterior samples
    posterior_df = pd.read_feather(posterior_feather_filepath)

    # Get the frequency range of the posterior samples
    f_min = posterior_df['Frequency'].min()
    f_max = posterior_df['Frequency'].max()
    # Filter injections based on frequency range
    injection_candidates = pd.DataFrame(injections_df[(injections_df['Frequency'] >= f_min) & (injections_df['Frequency'] <= f_max)])
    # Check if the injection candidates are empty
    if injection_candidates.empty:
        print("No injection candidates found within the frequency range of the posterior samples.")
        # Save injection candidates to a feather file
        if snr_cut is None:
            injection_candidates_filepath = f'./{candidates_feather_output_dir}/{posterior_feather_filepath.split("/")[-1].split("_")[0]}_candidates.feather'
            injection_candidates.to_feather(injection_candidates_filepath)
        return injection_candidates

    injection_candidates.reset_index(drop=True, inplace=True)
    # print(injection_candidates)

    # Save injection candidates to a feather file
    if snr_cut is None:
        injection_candidates_filepath = f'./{candidates_feather_output_dir}/{posterior_feather_filepath.split("/")[-1].split("_")[0]}_candidates.feather'
        injection_candidates.to_feather(injection_candidates_filepath)
    else:
        injection_candidates_filepath = f'./{candidates_feather_output_dir}/{posterior_feather_filepath.split("/")[-1].split("_")[0]}_candidates_snr_cut.feather'
        injection_candidates.to_feather(injection_candidates_filepath)

    return injection_candidates

In [ ]:
feather_files = glob.glob('./sangria_posterior_feathers/*.feather')
injections_df = pd.read_feather('/Users/aaron/Documents/lisa/catalog_metrics/data/ucb_params.feather')

In [ ]:
# if we do an SNR cut don't forget to filter the injections_df
injections_df = injections_df[injections_df['SNR'] >= 1]
for feather_file in feather_files:
    find_injection_candidates(injections_df, feather_file, 'posterior_injection_candidates_snr_cut', snr_cut=1)